In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd

In [ ]:
def create_df(l_col, l_per, l_val):
    df = pd.DataFrame([], columns=l_col)
    pointer=0
    
    for i in range(len(l_per)):
        list = [l_per[i]]
        pointer=i
        for j in range(len(l_col)-1):
            list.append(l_val[pointer].text)
            pointer = pointer + len(l_per)
        df.loc[len(df)] = list
    return df

def get_columns(yahoo_html_table):
    name_format = "^[A-Z]"
    list_col = []
    start_scraping = False
    
    for index, val in enumerate(yahoo_html_table):
        if (val.text == "Breakdown"): start_scraping = True
        if (val.text == "People Also Watch"): start_scraping = False
        if (start_scraping == True): 
            if (re.match(name_format, val.text)):
                list_col.append(val.text)
    return list_col

def get_periods(yahoo_html_table):
    date_format = "[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]"
    list_per = []
    start_scraping = False
    
    for index, val in enumerate(yahoo_html_table):
        if (val.text == "Breakdown"): start_scraping = True
        if (val.text == "People Also Watch"): start_scraping = False
        if (start_scraping == True): 
            if (val.text == "ttm" or re.match(date_format, val.text)): 
                list_per.append(val.text)
    return list_per

In [ ]:
driver = webdriver.Firefox()
wait_sec = 10
ticker = 'NVST'
url_income_statement = 'https://finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker

driver.get(url_income_statement)

#Click boton aceptar coockies
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
"/html/body/div/div/div/div/form/div[2]/div[2]/button[1]"))).click()

#Click boton registrarse luego
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
"/html/body/div[1]/div/div/div[1]/div/div[4]/div/div/div[1]/div/div/div/div/div/section/button[2]"))).click()

#Click boton Expandir datos
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
"/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/button/div/span"))).click()

html_income_statement = BeautifulSoup(driver.page_source, 'html.parser')

#Navegamos a balance-sheet
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
"a[href*='balance-sheet']"))).click()

#Click boton Expandir datos
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
"/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div/span"))).click()

html_balance_sheet = BeautifulSoup(driver.page_source, 'html.parser')

#Navegamos a cash-flow
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
"a[href*='cash-flow']"))).click()

#Click boton Expandir datos
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
"/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div/span"))).click()

html_cash_flow = BeautifulSoup(driver.page_source, 'html.parser')

#Navegamos a profile
WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
"a[href*='profile']"))).click()

#driver.quit()

In [ ]:
#Datos income_statement
income_statement_val = html_income_statement.findAll('div', attrs={"data-test":"fin-col"})
income_statement_col = html_income_statement.findAll('span')
df=create_df(get_columns(income_statement_col), get_periods(income_statement_col), income_statement_val)     
df

In [ ]:
#Datos Balance sheet
balance_sheet_val = html_balance_sheet.findAll('div', attrs={"data-test":"fin-col"})
balance_sheet_col = html_balance_sheet.findAll('span')
df=create_df(get_columns(balance_sheet_col), get_periods(balance_sheet_col), balance_sheet_val)     
df

In [ ]:
#Datos Cash flow
cash_flow_val = html_cash_flow.findAll('div', attrs={"data-test":"fin-col"})
cash_flow_col = html_cash_flow.findAll('span')
df=create_df(get_columns(cash_flow_col), get_periods(cash_flow_col), cash_flow_val)     
df

In [ ]:
#Datos Cash flow
cash_flow = html_cash_flow.findAll('span', attrs={"data-test":""})
start_scraping = False
for index, val in enumerate(cash_flow):
    print(val.text)  
    if (val.text == "Breakdown"): start_scraping = True
    if (val.text == "People Also Watch"): start_scraping = False
    if (start_scraping == True): print(val.text)   